# curating pr data

# Import general packages

In [25]:
import stitches as stitches


import pandas as pd
import pkg_resources
import xarray as xr
import numpy as np
import seaborn as sns

# Plotting options
sns.set(font_scale=1.3)
sns.set_style("white")
# For help with plotting
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 12, 6
pd.set_option('display.max_columns', None)

# import packages for spatial masking

In [26]:
import geopandas as gpd
# Spatial subsetting of netcdf files:
import regionmask

#  Set up time slices and area of interest (AOI) to focus on

- require ensemble average PR values over the ref period and comparison period
for an area of interest
- Do spatial aggregation for each ensemble member, take the time average in the
time window, calculate average across ensemble members

In [27]:
# Time slices
ref_start = '1995-01-01'
ref_end =  '2014-12-31'

comp_start = '2080-01-01'
comp_end =  '2099-12-31'

window_length = 20

In [28]:
# AOI
# working off https://www.earthdatascience.org/courses/use-data-open-source-python/hierarchical-data-formats-hdf/subset-netcdf4-climate-data-spatially-aoi/

# # physical land polygon files:
# url =  (    "https://naturalearth.s3.amazonaws.com/"
# "10m_physical/ne_10m_land.zip")

# # country URL
# url =  (    "https://naturalearth.s3.amazonaws.com/"
#             "10m_cultural/ne_10m_admin_0_countries.zip")

# state/province URL
url =  (    "https://naturalearth.s3.amazonaws.com/"
            "10m_cultural/ne_10m_admin_1_states_provinces.zip")


land_main_gdf = gpd.read_file(url)
land_main_gdf.head()

,featurecla,scalerank,adm1_code,diss_me,iso_3166_2,wikipedia,iso_a2,adm0_sr,name,name_alt,name_local,type,type_en,code_local,code_hasc,note,hasc_maybe,region,region_cod,provnum_ne,gadm_level,check_me,datarank,abbrev,postal,area_sqkm,sameascity,labelrank,name_len,mapcolor9,mapcolor13,fips,fips_alt,woe_id,woe_label,woe_name,latitude,longitude,sov_a3,adm0_a3,adm0_label,admin,geonunit,gu_a3,gn_id,gn_name,gns_id,gns_name,gn_level,gn_region,gn_a1_code,region_sub,sub_code,gns_level,gns_lang,gns_adm1,gns_region,min_label,max_label,min_zoom,wikidataid,name_ar,name_bn,name_de,name_en,name_es,name_fr,name_el,name_hi,name_hu,name_id,name_it,name_ja,name_ko,name_nl,name_pl,name_pt,name_ru,name_sv,name_tr,name_vi,name_zh,ne_id,name_he,name_uk,name_ur,name_fa,name_zht,FCLASS_ISO,FCLASS_US,FCLASS_FR,FCLASS_RU,FCLASS_ES,FCLASS_CN,FCLASS_TW,FCLASS_IN,FCLASS_NP,FCLASS_PK,FCLASS_DE,FCLASS_GB,FCLASS_BR,FCLASS_IL,FCLASS_PS,FCLASS_SA,FCLASS_EG,FCLASS_MA,FCLASS_PT,FCLASS_AR,FCLASS_JP,FCLASS_KO,FCLASS_VN,FCLASS_TR,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,FCLASS_TLC,geometry
0,Admin-1 states provinces,3,ARG-1309,1309,AR-E,None,AR,1,Entre Ríos,Entre-Rios,None,Provincia,Province,None,AR.ER,None,None,None,None,10.0,1,20,3,None,ER,0.0,-99,3,10,3,13,AR08,None,2344682.0,"Entre Rios, AR, Argentina",Entre Ríos,-32.02750,-59.2824,ARG,ARG,2,Argentina,Argentina,ARG,3434137.0,Provincia de Entre Rios,-988655.0,Entre Rios,1.0,None,AR.08,None,None,1.0,khm,AR08,None,6.0,11.0,6.0,Q44762,إنتري ريوس,এন্ত্রে রিও প্রদেশ,Entre Ríos,Entre Ríos,Entre Ríos,Entre Ríos,Έντρε Ρίος,एन्ट्रे रियोस,Entre Ríos,Entre Ríos,Entre Ríos,エントレ・リオス州,엔트레리오스,Entre Ríos,Entre Ríos,Entre Ríos,Энтре-Риос,Entre Ríos,Entre Ríos eyaleti,Entre Ríos,恩特雷里奥斯省,1159309789,אנטרה ריוס,Ентре-Ріос,صوبہ انترے ریوس,ایالت انتره ریوز,恩特雷里奥斯省,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((-58.20011 -32.44713, -58.20012 -32.4..."
1,Admin-1 states provinces,6,URY-8,8,UY-PA,None,UY,1,Paysandú,None,None,Departamento,Department,None,UY.PA,None,None,None,None,19.0,1,0,8,None,PA,0.0,-99,6,8,2,10,UY11,None,2347650.0,"PaysandÃº, UY, Uruguay",Paysandú,-32.09330,-57.2240,URY,URY,2,Uruguay,Uruguay,URY,3441242.0,Departamento de Paysandu,-908097.0,"Paysandu, Departamento de",1.0,None,UY.11,None,None,1.0,fra,UY11,None,8.0,11.0,8.0,Q16576,إدارة بايساندو,পেসান্ডো বিভাগ,Paysandú,Paysandú,Paysandú,Paysandú,Παϊσαντού,पयसंदु विभाग,Paysandú,Departemen Paysandú,dipartimento di Paysandú,パイサンドゥ県,파이산두,Paysandú,Paysandú,Paysandú,Пайсанду,Paysandú,Paysandu Departmanı,Paysandú,派桑杜省,1159307733,פאיסאנדו,Пайсанду,پایساندو محکمہ,بخش پایساندو,派桑杜省,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((-58.20012 -32.44720, -58.20011 -32.4..."
2,Admin-1 states provinces,2,IDN-1185,1185,ID-KI,None,ID,5,Kalimantan Timur,Kaltim,None,Propinsi,Province,None,ID.KI,None,None,None,None,15.0,1,20,1,None,KI,0.0,-99,2,16,6,11,ID14,None,2345723.0,"East Kalimantan, ID, Indonesia",Kalimantan Timur,1.28915,116.3540,IDN,IDN,2,Indonesia,Indonesia,IDN,1641897.0,Provinsi Kalimantan Timur,-2680740.0,"Kalimantan Timur, Provinsi",1.0,None,ID.14,None,None,1.0,ind,ID14,None,5.0,10.1,4.6,Q3899,كالمنتان الشرقية,পূর্ব কালিমান্তান,Ostkalimantan,East Kalimantan,Kalimantan Oriental,Kalimantan oriental,Ανατολικό Καλιμαντάν,पूर्व कालिमंतान,Kelet-Kalimantan,Kalimantan Timur,Kalimantan Orientale,東カリマンタン州,동칼리만탄,Oost-Kalimantan,Borneo Wschodnie,Kalimantan Oriental,Восточный Калимантан,Kalimantan Timur,Doğu Kalimantan,Đông Kalimantan,东加里曼丹省,1159310009,מזרח קלימנטאן,Східний Калімантан,مشرقی کالیمانتان,کالیمانتان شرقی,東加里曼丹省,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((1

In [29]:
# define conus as AOI
aoi = land_main_gdf[(land_main_gdf['admin'] == 'United States of America')&
                    (land_main_gdf['name'] != 'Hawaii') &
                     (land_main_gdf['name'] != 'Alaska') ].reset_index(drop=True).copy()

print(aoi.total_bounds)
# Get lat min, max
aoi_lat = [float(aoi.total_bounds[1]), float(aoi.total_bounds[3])]
aoi_lon = [float(aoi.total_bounds[0]), float(aoi.total_bounds[2])]
# The netcdf files use a global lat/lon so adjust values accordingly
aoi_lon[0] = aoi_lon[0] + 360
aoi_lon[1] = aoi_lon[1] + 360

aoi_lon, aoi_lat

[-124.73460724   24.54254792  -66.977325     49.36949412]


([235.2653927620001, 293.0226750010001],
 [24.54254791900007, 49.36949412145651])

# specify ESMs, variables, experiments

In [30]:
# The CMIP6 ESM we want to emulate and the variables we want to
# emulate


esm = ['CAMS-CSM1-0', 'MIROC6', 'GFDL-ESM4', 'FGOALS-g3',
'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0',
'ACCESS-ESM1-5', 'IPSL-CM6A-LR', 'CESM2-WACCM',
#'UKESM1-0-LL',
'CanESM5']
vars1 = ['pr']

exps = ['historical','ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp460', 'ssp585',
        'ssp434', 'ssp534-over']

# Pull pangeo dataframe with netcdf addresses for above

In [31]:
# pangeo table of ESMs for reference
pangeo_path = pkg_resources.resource_filename('stitches', 'data/pangeo_table.csv')
pangeo_data = pd.read_csv(pangeo_path)
# print(np.sort(pangeo_data.variable.unique()))
pangeo_data = pangeo_data[((
                               (pangeo_data['variable'].isin(vars1)) )  )
                          & ((pangeo_data['domain'].str.contains('mon')) ) &
                           ((pangeo_data['experiment'].isin(exps))) &
                           (pangeo_data['model'].isin(esm))].copy()

print(pangeo_data.head())


       model  experiment  ensemble variable  \
619   MIROC6  historical  r3i1p1f1       pr   
730   MIROC6  historical  r5i1p1f1       pr   
825   MIROC6  historical  r2i1p1f1       pr   
936   MIROC6  historical  r9i1p1f1       pr   
1049  MIROC6  historical  r8i1p1f1       pr   

                                                 zstore domain  
619   gs://cmip6/CMIP6/CMIP/MIROC/MIROC6/historical/...   Amon  
730   gs://cmip6/CMIP6/CMIP/MIROC/MIROC6/historical/...   Amon  
825   gs://cmip6/CMIP6/CMIP/MIROC/MIROC6/historical/...   Amon  
936   gs://cmip6/CMIP6/CMIP/MIROC/MIROC6/historical/...   Amon  
1049  gs://cmip6/CMIP6/CMIP/MIROC/MIROC6/historical/...   Amon  


# loop over files and do calculations

In [36]:
pr_holder = []


for esmname in esm[0:1]:
  for exp in exps[0:1]:

    print(esmname)
    print(exp)

    filelist = pangeo_data[(pangeo_data['model'] ==esmname) & (pangeo_data['experiment'] == exp)].copy()

    if not filelist.empty:
      print('notempty')
      ensemble_ds = []
      df_sum = 0
      n_good_files = 0
      for i in range(len(filelist)):
        print(i)
        print(exp)

        x = stitches.fx_pangeo.fetch_nc(filelist.iloc[i].zstore)
        if (i==0):
            aoi_mask = regionmask.mask_3D_geopandas(aoi,
                                                    x.lon,
                                                    x.lat)
        # # for testing if masking is happening right
        # x.sel(time=slice(ref_start, '1995-01-31')).pr.to_dataframe().reset_index().to_csv('test2.csv', index=False)
        #
        # x = x.sel(time=slice(ref_start, '1995-01-31'),
        # lon = slice(aoi_lon[0], aoi_lon[1]),
        # lat = slice(aoi_lat[0], aoi_lat[1])).where(aoi_mask).copy()
        # x1 = x.pr.to_dataframe().dropna().reset_index().drop(['region'], axis=1).reset_index(drop=True).copy()
        # x1.to_csv('test.csv', index=False)

        x = x.sel(lon = slice(aoi_lon[0], aoi_lon[1]),
                  lat = slice(aoi_lat[0], aoi_lat[1])).where(aoi_mask).copy()

        if (exp == 'historical'):
            print('in historical block')
            x = x.sel(time=slice(ref_start, ref_end)).copy()

            if (len(x.time) >= 12*window_length):
                # coerce to DF so we can properly lat weight to do spatial aggregation:
                x1 = x.pr.to_dataframe().dropna().reset_index().drop(['region'], axis=1).reset_index(drop=True).copy()

CAMS-CSM1-0
historical
notempty
0
historical
in historical block
1
historical
in historical block


In [49]:
monthly_aoi_pr = pd.DataFrame()
for name, group in x1.groupby('time'):
    lat = group['lat']
    area = np.cos(np.deg2rad(lat))
    df = pd.DataFrame({'time': group['time'].drop_duplicates()})
    df['aggregate_pr'] = sum(area * group['pr'])/sum(area)
    monthly_aoi_pr = monthly_aoi_pr.append(df)


monthly_aoi_pr

/var/folders/3t/b48tg9616bj3_bhcjwwll8cmg9cj39/T/ipykernel_45469/731026231.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  monthly_aoi_pr = monthly_aoi_pr.append(df)
/var/folders/3t/b48tg9616bj3_bhcjwwll8cmg9cj39/T/ipykernel_45469/731026231.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  monthly_aoi_pr = monthly_aoi_pr.append(df)
/var/folders/3t/b48tg9616bj3_bhcjwwll8cmg9cj39/T/ipykernel_45469/731026231.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  monthly_aoi_pr = monthly_aoi_pr.append(df)
/var/folders/3t/b48tg9616bj3_bhcjwwll8cmg9cj39/T/ipykernel_45469/731026231.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  monthly

,time,aggregate_pr
0,1995-01-16 12:00:00,0.000037
672,1995-02-15 00:00:00,0.000029
1344,1995-03-16 12:00:00,0.000028
2016,1995-04-16 00:00:00,0.000024
2688,1995-05-16 12:00:00,0.000029
...,...,...
157920,2014-08-16 12:00:00,0.000018
158592,2014-09-16 00:00:00,0.000019
159264,2014-10-16 12:00:00,0.000028
159936,2014-11-16 00:00:00,0.000026


In [35]:
                #spatial aggregation:

                # time average for this ensemble member:

                x_avg = x.mean('time').copy()
                x2 = x_avg['pr'].values.copy()
              df_sum = (x2 + df_sum).copy()
              n_good_files = n_good_files + 1
              if (i==0):
                  ensemble_ds = x_avg.copy()
            else:
                x=0
                x_avg=0



IndentationError: unindent does not match any outer indentation level (<tokenize>, line 10)

In [ ]:
        if (exp != 'historical'):
          print('in projection block')
          x = x.sel(time=slice(comp_start, comp_end)).copy()

          if (len(x.time) >= 12*window_length):
            x_avg = x.mean('time').copy()
            x2 = x_avg['pr'].values.copy()
            df_sum = (x2 + df_sum).copy()
            n_good_files = n_good_files + 1
            if (i==0):
              ensemble_ds = x_avg.copy()
          else:
            x=0
            x_avg=0

        if (exp == 'historical'):
          print('in historical block')
          x = x.sel(time=slice(ref_start, ref_end)).copy()

          if (len(x.time) >= 12*window_length):
              x_avg = x.mean('time').copy()
              x2 = x_avg['pr'].values.copy()
              df_sum = (x2 + df_sum).copy()
              n_good_files = n_good_files + 1
              if (i==0):
                  ensemble_ds = x_avg.copy()
            else:
                x=0
                x_avg=0


        del(x)
        del(x2)
        del(x_avg)

      df_ens_avg = (df_sum/n_good_files).copy()
      ensemble_ds['pr'].values = df_ens_avg.copy()

      # now that we have the ensemble, time window average for every grid
      # in the overall US block, further subset that to the actual conus shape
      # files
      conus_ensemble_ds = ensemble_ds.where(aoi).copy()

In [ ]:
conus_ensemble_ds = ensemble_ds.where(aoi)
conus_ensemble_ds
# # plot to make sure it's just the area of interest
# ensemble_ds['pr'].plot()
# plt.show()

In [ ]:
      # del(df_sum)
      # del(df_ens_avg)
      # del(n_good_files)
      #


      # # spatial masking
      # for j in range(len(r.map_info)):
      #   #print(j)
      #   #print(exp)
      #
      #   map_lon = r.map_info.iloc[j].lon +360
      #   map_lat = r.map_info.iloc[j].lat
      #
      #   extracted_cell = ensemble_ds.sel(lon = map_lon, lat = map_lat, method = 'nearest')
      #
      #   grid_val = extracted_cell.pr.values.copy()
      #   weight = np.cos(np.deg2rad(extracted_cell.lat.values))
      #
      #   df = pd.DataFrame({'esm':[esmname],
      #   'experiment':[exp],
      #   'map_lon': [map_lon],
      #   'map_lat': [map_lat],
      #   'data_lon': [float(extracted_cell.lon.values)],
      #   'data_lat': [float(extracted_cell.lat.values)],
      #   'ens_pr': [float(grid_val)],
      #   'area_weight':[weight]})
      #
      #   pr_holder = pr_holder.append(df)#pd.concat([pr_holder, df]).reset_index(drop = True).copy()
      #   print(pr_holder)
      #
      #   del(df)
      #   del(weight)
      #   del(grid_val)
      #   del(map_lon)
      #   del(map_lat)
      #   del(extracted_cell)

      #del(ensemble_ds)
pr_holder= pd.concat(pr_holder).reset_index(drop=True).copy()


In [ ]:
pangeo_path = pkg_resources.resource_filename('stitches', 'data/pangeo_table.csv')
pangeo_data = pd.read_csv(pangeo_path)
pangeo_data[['experiment']].drop_duplicates()

